In [1]:
import requests
from utils import EMDBRequestsException


In [7]:

emdbnumber = 1001
r = requests.get('https://www.ebi.ac.uk/emdb/api/entry/{}'.format(emdbnumber))


404

In [8]:
if r.status_code != 200:
    
    raise EMDBRequestsException('EMD_{} could not be accessed'.format(emdbnumber))

EMDBRequestsException: EMD_101 could not be accessed

In [16]:
search_term = 'natural_source_organism:"Apocrypta bakeri"'

r = requests.get('https://www.ebi.ac.uk/emdb/api/search/{}'.format(search_term))

In [2]:
import pandas as pd

In [59]:
import xmltodict
from pathlib import Path
import pprint
from collections import OrderedDict


def combine_keys(dic, key_check, value):
    if key_check in dic:
        dic[key_check] = f'{dic[key_check]},{value}'
    else:
        dic[key_check] = value
    
    return dic

                
def list_sorter(_list, final_dict, v=False):
    if all(isinstance(x, OrderedDict) for x in _list):
        for elem in _list:
            
            
            for nested_dict_key, nested_dict_value in elem.items():
                

                if isinstance(nested_dict_value, list):
                    # print(type(nested_dict_value))
                    list_sorter(elem[nested_dict_key], final_dict, v=True)
                    continue


                if not isinstance(nested_dict_value, str):
                    recursive_dict(elem, nested_dict_key, final_dict)
                else:
                    
                        
                    final_dict = combine_keys(
                        final_dict, 
                        nested_dict_key, 
                        nested_dict_value
                        )
    else:
        pass

def recursive_dict(dic,key, final_dict):
    """Function to break a structure dictionary into key-value pairs"""
    if dic[key] == None:
        return

    for k in dic[key]:
        if dic[key] == None:
            continue
            
        value_to_be_checked = dic[key][k]

        # Sometimes get lists of dicitonaries 
        if isinstance(value_to_be_checked, list):
            list_sorter(value_to_be_checked, final_dict)

        # Check to see if it's come to the end of a recursion
        elif not isinstance(value_to_be_checked, OrderedDict):
            final_dict = combine_keys(
                                final_dict, 
                                key, 
                                value_to_be_checked
                                )

            continue
        else:
            recursive_dict(dic[key], k, final_dict)
            
        
start=True
dict_before_dataframe = {}
for xml in Path('/mnt/sda1/luc/SnD/EmScraper/header').glob('*.xml'):
    name = xml.stem
    with open(xml) as xml:
        d = dict(xmltodict.parse(xml.read()))
        map_importantdetails = {name: {}}
        recursive_dict(d, 'emd', map_importantdetails[name])
    
        dict_before_dataframe |=  map_importantdetails
        



In [81]:
df = pd.DataFrame.from_dict(dict_before_dataframe).T

In [89]:
print(len(df))

# mask = df.resolution.str.contains('\d')
# mask = mask.fillna(False)

# df[df.resolution.isnull()].resolution
# df = df.reindex(df.index.sort_values(ascending=True))
# df = df.half_map
len(df[~df.file.isnull()].file)


24795


8522